## Feature Extraction Notebook
---

This notebook is used to extract features from the data. The audio is divided into windows of 1 second. The features extracted from each audio window are:
- **MFCC (Mel-Frequency Cepstral Coefficients)**: n_mfcc chosen by the user
- **Chroma STFT**: 12 coefficients
- **CQT (Constant-Q Transform)**: n_cqt chosen by the user
- **RMS (Root Mean Square) Energy**: 1 coefficient
- **Spectral Centroid**: 1 coefficient
- **Spectral Bandwidth**: 1 coefficient
- **Spectral Roll-off**: 1 coefficient
- **Zero Crossing Rate**: 1 coefficient

#### Sections:
- [Feature Extraction](#Feature-Extraction)
  - [Specific Features Extraction](#Specific-Features-Extraction)
  - [All Features Extraction](#All-Features-Extraction)

  
#### Findings:
- We have highly unbalanced classes in the dataset.

In [31]:
# import all the functions
import utils as utils
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
from tqdm.notebook import tqdm
import os
import importlib
import torchaudio
import torch

# import the utils module
importlib.reload(utils)

<module 'utils' from '/Users/andreaalberti/Desktop/Public_Projects/advanced-biomedical-project/notebooks/utils.py'>

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

# -------- tqdm DARK THEME --------

### Features Extraction

In [3]:
# hyperparameters
#sr = ''
sr = 4000

# set the paths to data
BASE_DIR = '../dataset/'
ARTIFACTS_DIR = BASE_DIR + f'artifacts_{sr}/'
EXTRAHLS_DIR = BASE_DIR + f'extrahls_{sr}/'
MURMURS_DIR = BASE_DIR + f'murmurs_{sr}/'
NORMALS_DIR = BASE_DIR + f'normals_{sr}/'
EXTRASTOLES_DIR = BASE_DIR + f'extrastoles_{sr}/'

# paths to save the features
FEATURES_RAW_DIR = '../features/raw/'
#FEATURES_RAW_DIR = "../features/balanced/priori/"

PATHS = [ARTIFACTS_DIR, EXTRAHLS_DIR, MURMURS_DIR, NORMALS_DIR, EXTRASTOLES_DIR]

# REMOVE THE AUGEMENTED SAMPLES WHEN EXTRACTING BASE RAW FEATURES
# for path in PATHS:
#      remove_generated_samples(path, 'USERAUGMENTED')

### Specific Features Extraction
- **MFCC (Mel-Frequency Cepstral Coefficients)**: n_mfcc chosen by the user
- **Chroma STFT**: 12 coefficients
- **CQT (Constant-Q Transform)**: n_cqt chosen by the user

In [42]:
def extract_features(
    dir_path: str,
    label: str,
    frame_length: float,
    sample_rate: int = 44100,
    n_mfcc: int = 13,
    melkwargs: dict = {},
    n_cqt: int = 84,
    get_mfcc: bool = True,
    get_chroma: bool = True,
    get_cqt: bool = True,
) -> list:
    """
    Extracts audio features (MFCC, chroma, RMS, spectral centroid, spectral bandwidth, spectral rolloff, zero-crossing rate)
    from audio files in the specified directory.

    Args:
        dir_path (str): The path to the directory containing audio files.
        label (str): The label associated with the extracted features.
        frame_length: The frame length for feature extraction.
        sample_rate (int, optional): The sample rate of the audio files. Defaults to 44100 Hz.
        n_mfcc (int, optional): The number of Mel-frequency cepstral coefficients (MFCCs) to extract. Defaults to 13.
        melkwargs (dict, optional): Additional arguments for Mel spectrogram computation.
        n_cqt (int, optional): The number of constant-Q transform (CQT) bins to extract. Defaults to 84.
        
    Returns:
        list: A list containing tensors of extracted features for each audio file.
    """
    filenames = os.listdir(dir_path)
    filenames = [file for file in filenames if not file.startswith(".")]
    frame_length = int(1 / frame_length)

    features = []  # List to store the features for all files

    for file in tqdm(filenames, desc=f"Extraction in progress"):
        audio, orig_sample_rate = torchaudio.load(os.path.join(dir_path, file))
        orig_sample_rate_tmp = int(orig_sample_rate / frame_length)
        audio_length = int(max(audio[0].shape) / (orig_sample_rate_tmp))

        sample_rate = orig_sample_rate

        # Reduce the audio from stereo to mono if needed
        if audio.shape[0] > 1:
            print(f"Converting stereo audio to mono for {file}...")
            audio = torch.mean(audio, dim=0).reshape(1, -1)

        features_local = []  # List to store the MFCC features for each file

        for i in range(audio_length):
            # Lazy load the audio, one sec at a time, to avoid memory issues
            audio_mono = utils.slicing(
                audio,
                offset=int(sample_rate / frame_length * i),
                num_frames=int(sample_rate / frame_length * (i + 1)),
            )

            # Get the MFCC features
            if get_mfcc:
                mfcc_features_tmp = utils.extract_mfcc(audio_mono, sample_rate, n_mfcc, melkwargs)
            if get_chroma:
                chroma_stft = utils.extract_chroma_stft(audio_mono, sample_rate)
            if get_cqt:
                cqt = utils.extract_cqt(audio_mono, sample_rate=sample_rate, n_cqt=n_cqt)

          #   rms = extract_rms(audio_mono)
          #   spec_cent = extract_spectral_centroid(audio_mono, sample_rate)
          #   spec_bw = extract_spectral_bandwidth(audio_mono, sample_rate)
          #   rolloff = extract_spectral_rolloff(audio_mono, sample_rate)
          #   zcr = extract_zero_crossing_rate(audio_mono)

            
            # Concatenate the features
            if get_mfcc:
                features_tmp = mfcc_features_tmp
            if get_chroma:
                features_tmp = chroma_stft
            if get_cqt:
                features_tmp = cqt
            
            if get_mfcc and get_chroma:
                features_tmp = torch.cat(
                    (mfcc_features_tmp, chroma_stft),
                    dim=0,
                )
            if get_mfcc and get_cqt:
                features_tmp = torch.cat(
                    (mfcc_features_tmp, cqt),
                    dim=0,
                )
            if get_chroma and get_cqt:
                features_tmp = torch.cat(
                    (chroma_stft, cqt),
                    dim=0,
                )
            if get_mfcc and get_chroma and get_cqt:
                features_tmp = torch.cat(
                    (mfcc_features_tmp, chroma_stft, cqt),
                    dim=0,
                )
            if not get_mfcc and not get_chroma and not get_cqt:
                features_tmp = torch.tensor([])

            features_local.append(features_tmp)


        # If features_local is empty, skip the file
        if features_local == []:
            print(f"No features extracted for {file}. Skipping...")
            continue

        # Stack the MFCC features into a single tensor
        features_local = torch.stack(features_local, dim=0)

        # Attach the label in the last column
        features_local = torch.cat(
            (features_local, torch.ones(features_local.shape[0], 1) * label), dim=1
        )

        # Attach the filename index in the last column
        features_local = torch.cat(
            (
                features_local,
                torch.full((features_local.shape[0], 1), filenames.index(file)),
            ),
            dim=1,
        )

        features.append(features_local)

    print("Finished processing all files.\n")
    return features

NameError: name 'features' is not defined

In [43]:
# ------------------------- MFCC -------------------------
n_mfcc_list = [30, 120]
names = ['artifacts', 'extrahls', 'murmurs', 'normals', 'extrastoles']
features_dict = {}
interval = 1
n_mfcc = 30
type_ = 'mfcc'  # mfcc, mel_spec, spec, cqt
# melkwargs = {'n_fft': 2048, 'hop_length': 512, 'n_mels': 128}

for n_mfcc in n_mfcc_list:
	# Save the features to a file
	storing_name = f'full_data_{interval}s_{sr}hz_{n_mfcc}{type_}'  ########## CHANGE THIS TO CHANGE THE NAME OF THE FILE


	if os.path.exists(FEATURES_RAW_DIR + storing_name + '.npy'):
		print('The features have already been extracted')
		
	else:
		for i, PATH_ in enumerate(PATHS):
			
			print(f'Extracting features from {PATH_}')
			features = extract_features(PATH_, label = i, frame_length = interval, n_mfcc=n_mfcc, get_mfcc=True, get_chroma=False, get_cqt=False)
			
			# Stack the features into a single tensor
			features = torch.cat(features, dim=0).numpy()
			print(f'The shape of the {PATH_} features tensor is: {features.shape}')
			
			X = features[:,:-2]
			y = features[:,-2]
			filename = features[:,-1]
			
			local_dict = {'X': X, 'y': y, 'filename': filename}
			
			features_dict[names[i]] = local_dict
				
		# Save the features to a file
		np.save(FEATURES_RAW_DIR + storing_name, features_dict)
				
		print('Features extracted and saved')





# ------------------------- CHROMA -------------------------









# ------------------------- CQT -------------------------



Extracting features from ../dataset/artifacts/


Extraction in progress:   0%|          | 0/84 [00:00<?, ?it/s]

Converting stereo audio to mono for artifact_2023_17.wav...
Converting stereo audio to mono for artifact_2023_16.wav...


/opt/homebrew/lib/python3.11/site-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


Converting stereo audio to mono for artifact_2023_14.wav...
Converting stereo audio to mono for artifact_2023_15.wav...
Converting stereo audio to mono for artifact_2023_39.wav...
Converting stereo audio to mono for artifact_2023_11.wav...
Converting stereo audio to mono for artifact_2023_38.wav...
Converting stereo audio to mono for artifact_2023_12.wav...
Converting stereo audio to mono for artifact_2023_13.wav...
Converting stereo audio to mono for artifact_2023_41.wav...
Converting stereo audio to mono for artifact_2023_40.wav...
Converting stereo audio to mono for artifact_2023_42.wav...
Converting stereo audio to mono for artifact_2023_43.wav...
Converting stereo audio to mono for artifact_2023_46.wav...
Converting stereo audio to mono for artifact_2023_44.wav...
Converting stereo audio to mono for artifact_2023_45.wav...
Converting stereo audio to mono for artifact_2023_36.wav...
Converting stereo audio to mono for artifact_2023_22.wav...
Converting stereo audio to mono for arti

Extraction in progress:   0%|          | 0/150 [00:00<?, ?it/s]

No features extracted for speed_13USERAUGMENTEDextrahls__201104021355.wav. Skipping...
No features extracted for pitch_44USERAUGMENTEDextrahls__201104021355.wav. Skipping...
No features extracted for speed_20USERAUGMENTEDextrahls__201104021355.wav. Skipping...
No features extracted for pitch_12USERAUGMENTEDextrahls__201104021355.wav. Skipping...
No features extracted for extrahls__201104021355.wav. Skipping...
No features extracted for noisy_3USERAUGMENTEDextrahls__201104021355.wav. Skipping...


KeyboardInterrupt: 

### All Features Extraction
- **MFCC (Mel-Frequency Cepstral Coefficients)**: n_mfcc chosen by the user
- **Chroma STFT**: 12 coefficients
- **CQT (Constant-Q Transform)**: n_cqt chosen by the user
- **RMS (Root Mean Square) Energy**: 1 coefficient
- **Spectral Centroid**: 1 coefficient
- **Spectral Bandwidth**: 1 coefficient
- **Spectral Roll-off**: 1 coefficient
- **Zero Crossing Rate**: 1 coefficient

In [53]:
# FEATURES EXTRACTION
names = ['artifacts', 'extrahls', 'murmurs', 'normals', 'extrastoles']
features_dict = {}
interval = 1
n_mfcc = 30
num_feat = 30
type_ = 'mfcc'  # mfcc, mel_spec, spec, cqt
# melkwargs = {'n_fft': 2048, 'hop_length': 512, 'n_mels': 128}

# Save the features to a file
storing_name = f'full_data_{interval}s_{sr}hz_{num_feat}{type_}'  ########## CHANGE THIS TO CHANGE THE NAME OF THE FILE


if os.path.exists(FEATURES_RAW_DIR + storing_name + '.npy'):
     print('The features have already been extracted')
     
else:
     for i, PATH_ in enumerate(PATHS):
          
          print(f'Extracting features from {PATH_}')
          features = extract_features(PATH_, label = i, frame_length = interval, n_mfcc=n_mfcc)
          
          # Stack the features into a single tensor
          features = torch.cat(features, dim=0).numpy()
          print(f'The shape of the {PATH_} features tensor is: {features.shape}')
          
          X = features[:,:-2]
          y = features[:,-2]
          filename = features[:,-1]
          
          local_dict = {'X': X, 'y': y, 'filename': filename}
          
          features_dict[names[i]] = local_dict
               
     # Save the features to a file
     np.save(FEATURES_RAW_DIR + storing_name, features_dict)
               
     print('Features extracted and saved')

          

Extracting features from ../dataset/artifacts/


Extraction in progress:   0%|          | 0/84 [00:00<?, ?it/s]

Converting stereo audio to mono for artifact_2023_17.wav...


/opt/homebrew/lib/python3.11/site-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


Converting stereo audio to mono for artifact_2023_16.wav...
Converting stereo audio to mono for artifact_2023_14.wav...


...

No features extracted for murmur__201104021355.wav. Skipping...
Converting stereo audio to mono for abnormal_s4_2023_2.wav...
No features extracted for murmur__171_1307971016233_E.wav. Skipping...
Converting stereo audio to mono for atrial_septal_defect_2023_6.wav...
Converting stereo audio to mono for abnormal_s3_2023_1.wav...
Converting stereo audio to mono for mitral_stenosis_2023_9.wav...
Converting stereo audio to mono for abnormal_s3_2023_0.wav...
Converting stereo audio to mono for holosystolic_murmur_2023_7.wav...
Converting stereo audio to mono for aortic_stenosis_2023_4.wav...
Converting stereo audio to mono for mitral_valve_prolapse_2023_10.wav...
Converting stereo audio to mono for innocent_murmur_2023_8.wav...
Finished processing all files.

The shape of the ../dataset/murmurs/ features tensor is: (1149, 28)
Extracting features from ../dataset/normals/


Extraction in progress:   0%|          | 0/355 [00:00<?, ?it/s]

No features extracted for normal__296_1311682952647_A1.wav. Skipping...
Converting stereo audio to mono for normal_2023_3.wav...
Converting stereo audio to mono for normal_2023_2.wav...
Converting stereo audio to mono for normal_2023_0.wav...
Converting stereo audio to mono for normal_2023_1.wav...
Finished processing all files.

The shape of the ../dataset/normals/ features tensor is: (2161, 28)
Extracting features from ../dataset/extrastoles/


Extraction in progress:   0%|          | 0/150 [00:00<?, ?it/s]

Finished processing all files.

The shape of the ../dataset/extrastoles/ features tensor is: (806, 28)
Features extracted and saved


In [54]:
# load the features and check the shape
features_dict = np.load(FEATURES_RAW_DIR + storing_name + '.npy', allow_pickle=True).item()
for key in features_dict.keys():
	print(f'The shape of the {key} features tensor is: {features_dict[key]["X"].shape}')
	print(f'The shape of the {key} labels tensor is: {features_dict[key]["y"].shape}')
	print(f'The shape of the {key} filenames tensor is: {features_dict[key]["filename"].shape}')
	print('-----------------------------------------------')

The shape of the artifacts features tensor is: (1199, 26)
The shape of the artifacts labels tensor is: (1199,)
The shape of the artifacts filenames tensor is: (1199,)
-----------------------------------------------
The shape of the extrahls features tensor is: (883, 26)
The shape of the extrahls labels tensor is: (883,)
The shape of the extrahls filenames tensor is: (883,)
-----------------------------------------------
The shape of the murmurs features tensor is: (1149, 26)
The shape of the murmurs labels tensor is: (1149,)
The shape of the murmurs filenames tensor is: (1149,)
-----------------------------------------------
The shape of the normals features tensor is: (2161, 26)
The shape of the normals labels tensor is: (2161,)
The shape of the normals filenames tensor is: (2161,)
-----------------------------------------------
The shape of the extrastoles features tensor is: (806, 26)
The shape of the extrastoles labels tensor is: (806,)
The shape of the extrastoles filenames tensor

In [55]:
s = 0
for key in features_dict.keys():
	s += features_dict[key]["X"].shape[0]
s

6198